In [1]:
import keras
from keras.datasets import mnist
from keras import backend as K
import numpy as np
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.


In [23]:
(x_train_origin, y_train_origin),(x_test_origin, y_test_origin) = mnist.load_data()

In [24]:
# if digit == 1, then label = 1, else label = 0
func = lambda x: 1 if x == 1 else 0

In [25]:
x_train = x_train_origin.reshape((60000, 28 * 28))
x_test = x_test_origin.reshape((10000, 28 * 28))

In [26]:
y_train = np.array([func(y) for y in y_train_origin])
y_test = np.array([func(y) for y in y_test_origin])

In [27]:
# Over Sampling, now the label for 1 is equal to 0
sm = SMOTE(random_state=12, ratio = 1.0)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [7]:
x_train = x_train.reshape(x_train.shape[0], -1).T
x_test = x_test.reshape(x_test.shape[0], -1).T

y_train = y_train.reshape(1,y_train.shape[0])
y_test = y_test.reshape(1,y_test.shape[0])

In [8]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [9]:
def sigmoid(z):
    
    s = 1.0/(1.0 + np.exp(-z))
    
    return s

In [10]:
def initialize_params_with_zeros(dim):
    
    w = np.zeros((dim,1))
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [11]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]

    A = sigmoid(np.dot(w.T, X) + b)
    # MSE Loss Function
    loss = (1/m) * np.sum(np.square(Y - A))
    
    dL = 2*(A - Y)
    dA = A * (1 - A) * dL
    
    # Derivative of w and d
    dw = (1/m) * np.dot(X, dA.T)
    db = (1/m) * np.sum(dA)

    grads = {"dw": dw,
             "db": db}
    return grads, loss

In [12]:
def optimize(w, b, X, Y, num_iters, batch_size, learning_rate, print_cost):
   
    costs = []
    m = X.shape[1]
    for i in range(num_iters):
        #cost_batch is used to collect the cost during one iteration over different batches
        cost_batch = []
        #stochastic gradient descent
        shuffled_indices = np.random.permutation(m)
        X_shuffled = X[:,shuffled_indices]
        y_shuffled = Y[:,shuffled_indices]
        #apply mini-batch
        for j in range(0, m, batch_size):
            x_batch = X_shuffled[:, j:j+batch_size]
            y_batch = y_shuffled[:, j:j+batch_size]
            grads, cost = propagate(w, b, x_batch, y_batch)
            dw = grads["dw"]
            db = grads["db"]
            w = w - learning_rate *  dw    
            b = b - learning_rate *  db
            cost_batch.append(cost)
            # the cost of one iteration is the average number over batches
            cost = np.mean(cost_batch)
        costs.append(cost)            
        if print_cost:
            print ("Cost after iteration %i: %f" % (i+1, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [13]:
def predict(w, b, X):
   
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    

    A = sigmoid(np.dot(w.T, X) + b)

    for i in range(A.shape[1]):
        if A[0][i] <= 0.5:
            Y_pred[0, i] = 0
        else:
            Y_pred[0, i] = 1
                
    assert(Y_pred.shape == (1, m))
    
    return Y_pred

In [14]:
def model(x_train, y_train, x_test, y_test, num_iters=20, batch_size = 5, learning_rate=0.5, print_cost=False):
    
    w, b = initialize_params_with_zeros(x_train.shape[0])                          

    parameters, grads, costs = optimize(w, b, x_train, y_train, num_iters, batch_size, learning_rate, print_cost)     

    w = parameters["w"]
    b = parameters["b"]
    

    Y_pred_test = predict(w, b, x_test)
    Y_pred_train = predict(w, b, x_train)

    print("")
    
    train_accuracy =  100 - np.mean(np.abs(Y_pred_train - y_train)) * 100
    test_accuracy = 100 - np.mean(np.abs(Y_pred_test - y_test)) * 100
    
    print("train accuracy: {} %".format(train_accuracy))
    print("test accuracy: {} %".format(test_accuracy))
    repr('train accuracy')
    
    d = {"costs": costs,
         "Y_pred_test": Y_pred_test, 
         "Y_pred_train" : Y_pred_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iters": num_iters}
    
    return d, train_accuracy, test_accuracy

In [15]:
# the accuracy for number 1
d, train, test = model(x_train, y_train, x_test, y_test, num_iters=10, batch_size=10000, learning_rate=0.5, print_cost=True)

Cost after iteration 1: 0.094737
Cost after iteration 2: 0.044217
Cost after iteration 3: 0.036695
Cost after iteration 4: 0.032951
Cost after iteration 5: 0.030714
Cost after iteration 6: 0.029047
Cost after iteration 7: 0.027824
Cost after iteration 8: 0.026891
Cost after iteration 9: 0.026011
Cost after iteration 10: 0.025435

train accuracy: 97.25675015960043 %
test accuracy: 97.63 %


In [21]:
# the accuracy for number 0 to number 9

for i in range(10):
    
    x_train = x_train_origin.reshape((60000, 28 * 28))
    x_test = x_test_origin.reshape((10000, 28 * 28))
    
    # if digit == i, then label = 1, else label = 0
    func = lambda x: 1 if x == i else 0
    y_train = np.array([func(y) for y in y_train_origin])
    y_test = np.array([func(y) for y in y_test_origin])
    
    # Over Sampling, now the label for 1 is equal to 0
    sm = SMOTE(random_state=12, ratio = 1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    
    
    x_train = x_train.reshape(x_train.shape[0], -1).T
    x_test = x_test.reshape(x_test.shape[0], -1).T

    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    
    y_train = y_train.reshape(1,y_train.shape[0])
    y_test = y_test.reshape(1,y_test.shape[0])
    
    
    print("\n", "the accuracy of number {}".format(i))
    
    d, train, test = model(x_train, y_train, x_test, y_test, num_iters=10, batch_size=10000, learning_rate=0.5, print_cost=False)
    



 the accuracy of number 0

train accuracy: 97.42034506352053 %
test accuracy: 97.22 %

 the accuracy of number 1

train accuracy: 97.26238311615157 %
test accuracy: 97.63 %

 the accuracy of number 2

train accuracy: 93.98060767551164 %
test accuracy: 95.01 %

 the accuracy of number 3

train accuracy: 92.49382761885315 %
test accuracy: 92.52 %

 the accuracy of number 4

train accuracy: 94.9130322390044 %
test accuracy: 94.26 %

 the accuracy of number 5

train accuracy: 91.58192711482438 %
test accuracy: 90.78 %

 the accuracy of number 6

train accuracy: 96.7059280352058 %
test accuracy: 96.1 %

 the accuracy of number 7

train accuracy: 96.26314320275425 %
test accuracy: 96.64 %

 the accuracy of number 8

train accuracy: 88.282332083695 %
test accuracy: 87.22 %

 the accuracy of number 9

train accuracy: 91.35261142254538 %
test accuracy: 89.03 %
